In [117]:
'''
Course Name - Soft Computing Lab
Course No. - CSE 4238
Name - Simon Das
Section - C2
Student ID - 16.01.04.146
'''

'\nCourse Name - Soft Computing Lab\nCourse No. - CSE 4238\nName - Simon Das\nSection - C2\nStudent ID - 16.01.04.146\n'

In [118]:
from google.colab import files
print("upload the train file : ")
uploaded_train = files.upload()
print("\n\nupload the test file : ")
uploaded_test = files.upload()

upload the train file : 


Saving training_Pyramid_1000.data.txt to training_Pyramid_1000.data (2).txt


upload the test file : 


Saving testing_Pyramid_1000.data.txt to testing_Pyramid_1000.data (2).txt


LOAD INPUT
HERE SAMPLE_DATASET TAKES TRAINING DATA AS AN INPUT

In [0]:
def input_data(sample_dataset):
  trainData = pd.read_csv(io.BytesIO(uploaded_train[sample_dataset]), sep = '\t', header = None)
  return trainData

FOR EMISSION MATRIX
>CALCULATE THE PROBABILITY OF DIFFERENT TEST SAMPLE AND STORED IN EMMISION MATRIX

In [0]:
def emission_matrix_calculate(trainData):
  fair_cheat_win_lose = trainData.groupby(0)[1].value_counts()
  
  fair_win = round(fair_cheat_win_lose['fair']['win'] / fair_cheat_win_lose['fair'].sum(), 2)
  fair_lose = round(fair_cheat_win_lose['fair']['lose'] / fair_cheat_win_lose['fair'].sum(), 2)
  cheat_win = round(fair_cheat_win_lose['cheat']['win'] / fair_cheat_win_lose['cheat'].sum(), 2)
  cheat_lose = round(fair_cheat_win_lose['cheat']['lose'] / fair_cheat_win_lose['cheat'].sum(), 2)

  emission_matrix = np.array([[fair_win, fair_lose], [cheat_win, cheat_lose]])

  return emission_matrix

FOR TRANSITION MATRIX
> CALCULATE THE PROBABILITY OF DIFFERENT STATE TRANSITION FOR TRAINING DATA
 AND VALUE STORED IN TRANSITION MATRIX FOR FURTHER CALCULATION


In [0]:
def transition_matrix_count(trainData):
  fair_fair, fair_cheat, cheat_fair, cheat_cheat = 0, 0, 0, 0
  fair_fair_pro, fair_cheat_pro, cheat_fair_pro, cheat_cheat_pro = 0, 0, 0, 0

  for i in range(len(trainData) - 1):
    if trainData[0][i] == 'fair':
      if trainData[0][i] == trainData[0][i+1]:
        fair_fair += 1
      else:
        fair_cheat += 1
    elif trainData[0][i] == 'cheat':
      if trainData[0][i] == trainData[0][i+1]:
        cheat_cheat += 1
      else:
        cheat_fair += 1

  fair_fair_pro = round((fair_fair / (fair_fair + fair_cheat)), 2)
  fair_cheat_pro = round((fair_cheat / (fair_fair + fair_cheat)), 2)
  cheat_cheat_pro = round((cheat_cheat / (cheat_cheat + cheat_fair)), 2)
  cheat_fair_pro = round((cheat_fair / (cheat_cheat + cheat_fair)), 2)

  transition_matrix = np.array([[fair_fair_pro, fair_cheat_pro], [cheat_fair_pro, cheat_cheat_pro]])

  return transition_matrix

INPUT_OBSERVATION FUNCTION TAKES TESTING DATA SET

In [0]:
import pandas as pd
import io

def input_observation(testingData):
  testing_data = pd.read_csv(io.BytesIO(uploaded_test[testingData]), sep = '\t', header=None)
  return testing_data

IMPLEMENT HMM USING THE VITERBI ALGORITHM
>Reference:
Murphy, Kevin P. Machine Learning: A Probabilistic Perspective. MIT press, 2012.

![alt text](https://drive.google.com/uc?id=1J9yuHGxAKhbX67x_lmFylPzeZAKkm7Ns)

In [0]:
def viterbi(observations, emission_matrix, initial_pro, transition_matrix):
  fair_nodes, cheat_nodes, final_output, fair_counter, cheat_counter = [], [], [], 0, 0

  if observations[0][0] == "win":
    fair_nodes.append(initial_pro[0] * emission_matrix[0][0])
    cheat_nodes.append(initial_pro[1] * emission_matrix[1][0])
  else:
    fair_nodes.append(initial_pro[0] * emission_matrix[0][1])
    cheat_nodes.append(initial_pro[1] * emission_matrix[1][1])

  if fair_nodes[0] >= cheat_nodes[0]:
    final_output.append('fair')
    fair_counter += 1
  else:
    final_output.append('cheat')
    cheat_counter += 1

  for i in range(1, len(observations)):
    if observations[0][i] == 'win':
      fair_nodes.append(max(np.log(emission_matrix[0][0]) * np.log(transition_matrix[0][0]) * fair_nodes[i-1], np.log(emission_matrix[0][0]) * np.log(transition_matrix[1][0]) * cheat_nodes[i-1]))
      cheat_nodes.append(max(np.log(emission_matrix[1][0]) * np.log(transition_matrix[1][1]) * cheat_nodes[i-1], np.log(emission_matrix[1][0]) * np.log(transition_matrix[0][1]) * fair_nodes[i-1]))
    else:
      fair_nodes.append(max(np.log(emission_matrix[0][1]) * np.log(transition_matrix[0][0]) * fair_nodes[i-1], np.log(emission_matrix[0][1]) * np.log(transition_matrix[1][0]) * cheat_nodes[i-1]))
      cheat_nodes.append(max(np.log(emission_matrix[1][1]) * np.log(transition_matrix[1][1]) * cheat_nodes[i-1], np.log(emission_matrix[1][1]) * np.log(transition_matrix[0][1]) * fair_nodes[i-1]))

    if fair_nodes[i] >= cheat_nodes[i]:
      final_output.append('fair')
      fair_counter += 1
    else:
      final_output.append('cheat')
      cheat_counter += 1

  score = [fair_counter, cheat_counter]

  return score, final_output


MAIN FUNCTION

In [124]:
import numpy as np

def main():

    sample_dataset = "training_Pyramid_1000.data.txt"  
    testing_data = "testing_Pyramid_1000.data.txt"     

    observations = input_observation(testing_data)
    initial_pro = np.array([0.5, 0.5])

    trainData = input_data(sample_dataset)

    emission_matrix = emission_matrix_calculate(trainData)
    transition_matrix = transition_matrix_count(trainData)

    print("Transition Matrix\n", transition_matrix)
    print("Emission Matrix\n", emission_matrix)

    score, final_output = viterbi(observations, emission_matrix, initial_pro, transition_matrix)
    print(score)
    print(final_output)

main()

Transition Matrix
 [[0.72 0.28]
 [0.29 0.71]]
Emission Matrix
 [[0.54 0.46]
 [0.3  0.7 ]]
[513, 487]
['cheat', 'fair', 'cheat', 'fair', 'cheat', 'fair', 'cheat', 'fair', 'cheat', 'cheat', 'fair', 'cheat', 'fair', 'cheat', 'fair', 'cheat', 'fair', 'fair', 'fair', 'cheat', 'fair', 'fair', 'fair', 'cheat', 'fair', 'cheat', 'fair', 'cheat', 'fair', 'cheat', 'fair', 'cheat', 'fair', 'cheat', 'fair', 'cheat', 'fair', 'cheat', 'fair', 'fair', 'fair', 'fair', 'cheat', 'fair', 'cheat', 'cheat', 'fair', 'cheat', 'fair', 'cheat', 'fair', 'fair', 'cheat', 'fair', 'fair', 'fair', 'fair', 'fair', 'fair', 'fair', 'cheat', 'fair', 'cheat', 'cheat', 'fair', 'cheat', 'cheat', 'cheat', 'fair', 'cheat', 'cheat', 'cheat', 'fair', 'cheat', 'fair', 'cheat', 'cheat', 'cheat', 'fair', 'cheat', 'fair', 'cheat', 'cheat', 'fair', 'cheat', 'fair', 'cheat', 'fair', 'cheat', 'fair', 'cheat', 'fair', 'fair', 'fair', 'cheat', 'fair', 'fair', 'fair', 'fair', 'fair', 'cheat', 'fair', 'cheat', 'fair', 'cheat', 'fair', 'c